# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# Import API key
from config import g_key

### Store WeatherPy results into DataFrame

In [2]:
df = pd.read_csv("../Outputs/WeatherPy/Raw_Data.csv")
df = df.drop("Unnamed: 0",axis=1)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pevek,69.70,170.31,54.82,66,89,9.44,RU,1595802360
1,Quang Ngai,15.12,108.80,75.87,87,8,4.29,VN,1595802496
2,Laas,46.62,10.70,57.99,93,51,3.36,IT,1595802496
3,Isangel,-19.55,169.27,77.00,83,40,11.41,VU,1595802273
4,Bluff,-46.60,168.33,39.00,83,0,3.00,NZ,1595802229


### Humidity Heatmap
* Configure gmaps.
* Using the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = df[["Lat","Lng"]]
weights = df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Find ideal vacation destinations based on weather
* Narrow down the cities to fit weather conditions.
* Add a "Hotel Name" column to the DataFrame.
* Store into variable named `hotel_df`.

In [5]:
hotel_df = df.loc[(df["Max Temp"]<=85) & (df["Max Temp"]>=70) & (df["Humidity"]<=30) & (df["Cloudiness"]<=20),:]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
146,Bilibino,68.05,166.44,79.65,29,5,24.11,RU,1595802360,
194,Zatoka,46.07,30.47,82.00,28,20,11.18,UA,1595802567,
306,Salihli,38.48,28.15,77.00,10,0,58.00,TR,1595802609,
340,Kumul,42.80,93.45,82.17,19,0,18.12,CN,1595802396,
485,Bozeman,45.68,-111.04,82.99,28,1,6.93,US,1595802678,
524,Ubari,26.59,12.78,82.67,17,0,6.93,LY,1595802692,


### Hotel Map
* Set parameters to search for hotels with 5000 meters of choice locations.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
radius = 5000
keyword = "hotel"
base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&"

# Find hotel information at desired locations
for index, row in hotel_df.iterrows():
    response = requests.get(f"{base_url}location={f'{row.Lat},{row.Lng}'}&radius={radius}&keyword={keyword}")
    try:
        hotel_df.loc[index,"Hotel Name"] = response.json()["results"][0]["name"]
    except:
        print(f"No hotel found for location: {row['City']}")
        
# Eliminate locations without any hotel results
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]

No hotel found for location: Ubari


In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
146,Bilibino,68.05,166.44,79.65,29,5,24.11,RU,1595802360,Profilaktoriy Baes
194,Zatoka,46.07,30.47,82.00,28,20,11.18,UA,1595802567,Prybii
306,Salihli,38.48,28.15,77.00,10,0,58.00,TR,1595802609,Hotel Lidya Sardes
340,Kumul,42.80,93.45,82.17,19,0,18.12,CN,1595802396,Back Hotel
485,Bozeman,45.68,-111.04,82.99,28,1,6.93,US,1595802678,Holiday Inn Bozeman


In [8]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
locations = df[["Lat","Lng"]]
weights = df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100,point_radius = 3)
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))